# Deploying Serverless Endpoints From SageMaker Model Registry

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/serverless-inference|serverless-model-registry.ipynb)

---

## SageMaker XGBoost Algorithm Regression Example

Amazon SageMaker Serverless Inference is a purpose-built inference option that makes it easy for customers to deploy and scale ML models. Serverless Inference is ideal for workloads which have idle periods between traffic spurts and can tolerate cold starts. Serverless endpoints also automatically launch compute resources and scale them in and out depending on traffic, eliminating the need to choose instance types or manage scaling policies.

[SageMaker Model Registry](https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry.html) can be used to catalog and manage different model versions. Model Registry now supports deploying registered models to serverless endpoints. For this notebook we will take the existing [XGBoost Serverless example](https://github.com/aws/amazon-sagemaker-examples/blob/main/serverless-inference/Serverless-Inference-Walkthrough.ipynb) and integrate with the Model Registry. From there we will take our trained model and deploy it to a serverless endpoint using the Boto3 Python SDK. Note that there is no support for Model Registry in the SageMaker SDK with serverless endpoints at the moment.

<b>Notebook Setting</b>
- <b>SageMaker Studio</b>: Python 3 (Data Science)
- <b>Regions Available</b>: SageMaker Serverless Inference is currently available in the following regions in preview: US East (Northern Virginia), US East (Ohio), US West (Oregon), EU (Ireland), Asia Pacific (Tokyo) and Asia Pacific (Sydney). After general availability it should be available in all commercial regions. To verify availability stay up to date with the [documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints.html) which will reflect all supported regions.

## Table of Contents
- Setup
- Model Training
- Model Registry
- Deployment
    - Model Creation
    - Endpoint Configuration Creation
    - Serverless Endpoint Creation
    - Endpoint Invocation
- Cleanup

## Setup

For testing you need to properly configure your Notebook Role to have <b>SageMaker Full Access</b>.

In [1]:
! pip install sagemaker botocore boto3 awscli --upgrade

In [2]:
# Setup clients
import boto3

client = boto3.client(service_name="sagemaker")
runtime = boto3.client(service_name="sagemaker-runtime")

In [3]:
import sagemaker
from sagemaker.estimator import Estimator

boto_session = boto3.session.Session()
region = boto_session.region_name
print(region)

sagemaker_session = sagemaker.Session()
base_job_prefix = "xgboost-example"
role = sagemaker.get_execution_role()
print(role)

default_bucket = sagemaker_session.default_bucket()
s3_prefix = base_job_prefix

training_instance_type = "ml.m5.xlarge"

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/20/25 18:12:59] INFO     Found credentials from IAM Role:                                   ]8;id=689111;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=978413;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
us-east-2
arn:aws:iam::160885266928:role/service-role/AmazonSageMakerServiceCatalogProductsUseRole


In [4]:
# retrieve data
s3 = boto3.client("s3")
s3.download_file(
    f"sagemaker-example-files-prod-{region}",
    "datasets/tabular/uci_abalone/train_csv/abalone_dataset1_train.csv",
    "abalone_dataset1_train.csv",
)

In [5]:
# upload data to S3
!aws s3 cp abalone_dataset1_train.csv s3://{default_bucket}/xgboost-regression/train.csv

upload: ./abalone_dataset1_train.csv to s3://sagemaker-us-east-2-160885266928/xgboost-regression/train.csv


## Model Training

Now, we train an ML model using the [SageMaker XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html). In this example, we use a SageMaker-provided XGBoost container image and configure an estimator to train our model.

In [6]:
from sagemaker.inputs import TrainingInput

training_path = f"s3://{default_bucket}/xgboost-regression/train.csv"
train_input = TrainingInput(training_path, content_type="text/csv")

In [7]:
model_path = f"s3://{default_bucket}/{s3_prefix}/xgb_model"

# retrieve xgboost image
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=region,
    version="1.0-1",
    py_version="py3",
    instance_type=training_instance_type,
)

# Configure Training Estimator
xgb_train = Estimator(
    image_uri=image_uri,
    instance_type=training_instance_type,
    instance_count=1,
    output_path=model_path,
    sagemaker_session=sagemaker_session,
    role=role,
)

# Set Hyperparameters
xgb_train.set_hyperparameters(
    objective="reg:linear",
    num_round=50,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.7,
    silent=0,
)

In [8]:
# Fit model
xgb_train.fit({"train": train_input})

[03/20/25 18:13:28] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=114734;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=444123;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[03/20/25 18:13:29] INFO     Creating training-job with name:                                       ]8;id=621744;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=476153;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-03-20-18-13-29-015                                             

2025-03-20 18:13:31 Starting - Starting the training job...
..25-03-20 18:14:02 Downloading - Downloading input data.
..25-03-20 18:14:27 Downloading - Downloading the training image.
.[2025-03-20 18:15:15.703 ip-10-0-193-239.us-east-2.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:linear to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
[18:15:15] 2923x8 matrix with 23384 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-03-20 18:15:15.765 ip-10-0-193-239.us-east-2.compute.internal:7 INFO js

In [9]:
# Retrieve model data from training job
model_artifacts = xgb_train.model_data
model_artifacts

's3://sagemaker-us-east-2-160885266928/xgboost-example/xgb_model/sagemaker-xgboost-2025-03-20-18-13-29-015/output/model.tar.gz'

## Model Registry

In [10]:
# Create a Model Package Group: https://docs.aws.amazon.com/sagemaker/latest/dg/model-registry-model-group.html
import time
from time import gmtime, strftime

model_package_group_name = "xgboost-abalone" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_package_group_input_dict = {
    "ModelPackageGroupName": model_package_group_name,
    "ModelPackageGroupDescription": "Model package group for xgboost regression model with Abalone dataset",
}

create_model_pacakge_group_response = client.create_model_package_group(
    **model_package_group_input_dict
)
print(
    "ModelPackageGroup Arn : {}".format(create_model_pacakge_group_response["ModelPackageGroupArn"])
)

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-2:160885266928:model-package-group/xgboost-abalone2025-03-20-18-15-53


In [11]:
model_package_group_arn = create_model_pacakge_group_response["ModelPackageGroupArn"]
modelpackage_inference_specification = {
    "InferenceSpecification": {
        "Containers": [
            {
                "Image": image_uri,
            }
        ],
        "SupportedContentTypes": ["text/csv"],
        "SupportedResponseMIMETypes": ["text/csv"],
    }
}

# Specify the model source
model_url = model_artifacts

# Specify the model data
modelpackage_inference_specification["InferenceSpecification"]["Containers"][0][
    "ModelDataUrl"
] = model_url

create_model_package_input_dict = {
    "ModelPackageGroupName": model_package_group_arn,
    "ModelPackageDescription": "Model for regression with the Abalone dataset",
    "ModelApprovalStatus": "PendingManualApproval",
}
create_model_package_input_dict.update(modelpackage_inference_specification)

# Create cross-account model package
create_mode_package_response = client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print("ModelPackage Version ARN : {}".format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-2:160885266928:model-package/xgboost-abalone2025-03-20-18-15-53/1


In [12]:
client.list_model_packages(ModelPackageGroupName=model_package_group_name)

{'ModelPackageSummaryList': [{'ModelPackageGroupName': 'xgboost-abalone2025-03-20-18-15-53',
   'ModelPackageVersion': 1,
   'ModelPackageArn': 'arn:aws:sagemaker:us-east-2:160885266928:model-package/xgboost-abalone2025-03-20-18-15-53/1',
   'ModelPackageDescription': 'Model for regression with the Abalone dataset',
   'CreationTime': datetime.datetime(2025, 3, 20, 18, 15, 57, 432000, tzinfo=tzlocal()),
   'ModelPackageStatus': 'Completed',
   'ModelApprovalStatus': 'PendingManualApproval'}],
 'ResponseMetadata': {'RequestId': 'e8f4f3c6-04ae-4a83-bb15-8aaf8280b269',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e8f4f3c6-04ae-4a83-bb15-8aaf8280b269',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '413',
   'date': 'Thu, 20 Mar 2025 18:16:16 GMT'},
  'RetryAttempts': 0}}

In [13]:
model_package_arn = client.list_model_packages(ModelPackageGroupName=model_package_group_name)[
    "ModelPackageSummaryList"
][0]["ModelPackageArn"]
model_package_arn

'arn:aws:sagemaker:us-east-2:160885266928:model-package/xgboost-abalone2025-03-20-18-15-53/1'

In [14]:
client.describe_model_package(ModelPackageName=model_package_arn)

{'ModelPackageGroupName': 'xgboost-abalone2025-03-20-18-15-53',
 'ModelPackageVersion': 1,
 'ModelPackageArn': 'arn:aws:sagemaker:us-east-2:160885266928:model-package/xgboost-abalone2025-03-20-18-15-53/1',
 'ModelPackageDescription': 'Model for regression with the Abalone dataset',
 'CreationTime': datetime.datetime(2025, 3, 20, 18, 15, 57, 432000, tzinfo=tzlocal()),
 'InferenceSpecification': {'Containers': [{'Image': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost:1.0-1-cpu-py3',
    'ImageDigest': 'sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
    'ModelDataUrl': 's3://sagemaker-us-east-2-160885266928/xgboost-example/xgb_model/sagemaker-xgboost-2025-03-20-18-13-29-015/output/model.tar.gz',
    'ModelDataETag': '74547ed767af996b7fcb78944663c320'}],
  'SupportedContentTypes': ['text/csv'],
  'SupportedResponseMIMETypes': ['text/csv']},
 'ModelPackageStatus': 'Completed',
 'ModelPackageStatusDetails': {'ValidationStatuses': [],
  'ImageScanSt

In [15]:
# Approve the model package
model_package_update_input_dict = {
    "ModelPackageArn": model_package_arn,
    "ModelApprovalStatus": "Approved",
}
model_package_update_response = client.update_model_package(**model_package_update_input_dict)
print(model_package_update_response)

{'ModelPackageArn': 'arn:aws:sagemaker:us-east-2:160885266928:model-package/xgboost-abalone2025-03-20-18-15-53/1', 'ResponseMetadata': {'RequestId': '9885de0b-110c-4d86-9223-21f362c23dc9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '9885de0b-110c-4d86-9223-21f362c23dc9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '113', 'date': 'Thu, 20 Mar 2025 18:17:47 GMT'}, 'RetryAttempts': 0}}


## Deployment

### Model Creation

In [16]:
model_name = "xgboost-serverless-model" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("Model name : {}".format(model_name))
container_list = [{"ModelPackageName": model_package_arn}]

create_model_response = client.create_model(
    ModelName=model_name, ExecutionRoleArn=role, Containers=container_list
)
print("Model arn : {}".format(create_model_response["ModelArn"]))

Model name : xgboost-serverless-model2025-03-20-18-17-51
Model arn : arn:aws:sagemaker:us-east-2:160885266928:model/xgboost-serverless-model2025-03-20-18-17-51


### Endpoint Configuration Creation
This is where you can adjust the [Serverless Configuration](https://docs.aws.amazon.com/sagemaker/latest/dg/serverless-endpoints-create.html) for your endpoint. The current max concurrent invocations for a single endpoint, known as `MaxConcurrency`, can be any value from 1 to 200, and `MemorySize` can be any of the following: 1024 MB, 2048 MB, 3072 MB, 4096 MB, 5120 MB, or 6144 MB.

In [17]:
endpoint_config_name = "xgboost-serverless-epc" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print(endpoint_config_name)
create_endpoint_config_response = client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "ServerlessConfig": {"MemorySizeInMB": 1024, "MaxConcurrency": 10},
            "ModelName": model_name,
            "VariantName": "AllTraffic",
        }
    ],
)
print("Endpoint Configuration Arn: " + create_endpoint_config_response["EndpointConfigArn"])

xgboost-serverless-epc2025-03-20-18-18-06
Endpoint Configuration Arn: arn:aws:sagemaker:us-east-2:160885266928:endpoint-config/xgboost-serverless-epc2025-03-20-18-18-06


### Endpoint Creation
Now that we have an endpoint configuration, we can create a serverless endpoint and deploy our model to it. When creating the endpoint, provide the name of your endpoint configuration and a name for the new endpoint.

In [18]:
endpoint_name = "xgboost-serverless-ep" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = client.create_endpoint(
    EndpointName=endpoint_name, EndpointConfigName=endpoint_config_name
)
print(create_endpoint_response["EndpointArn"])

EndpointName=xgboost-serverless-ep2025-03-20-18-18-10
arn:aws:sagemaker:us-east-2:160885266928:endpoint/xgboost-serverless-ep2025-03-20-18-18-10


Wait until the endpoint status is `InService` before invoking the endpoint.

In [19]:
import time

describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)

while describe_endpoint_response["EndpointStatus"] == "Creating":
    describe_endpoint_response = client.describe_endpoint(EndpointName=endpoint_name)
    print(describe_endpoint_response["EndpointStatus"])
    time.sleep(15)

describe_endpoint_response

Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
Creating
InService


{'EndpointName': 'xgboost-serverless-ep2025-03-20-18-18-10',
 'EndpointArn': 'arn:aws:sagemaker:us-east-2:160885266928:endpoint/xgboost-serverless-ep2025-03-20-18-18-10',
 'EndpointConfigName': 'xgboost-serverless-epc2025-03-20-18-18-06',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
     'ResolvedImage': '257758044811.dkr.ecr.us-east-2.amazonaws.com/sagemaker-xgboost@sha256:da43a3b51e4fddd7743132d10eb2578d42c33f1a4d256bb4eaad349d4515b9b7',
     'ResolutionTime': datetime.datetime(2025, 3, 20, 18, 18, 11, 830000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 0,
   'CurrentServerlessConfig': {'MemorySizeInMB': 1024, 'MaxConcurrency': 10}}],
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2025, 3, 20, 18, 18, 11, 398000, tzinfo=tzloca

### Invocation
Invoke the endpoint by sending a request to it. The following is a sample data point grabbed from the CSV file downloaded from the public Abalone dataset.

In [20]:
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=b".345,0.224414,.131102,0.042329,.279923,-0.110329,-0.099358,0.0",
    ContentType="text/csv",
)

print(response["Body"].read())

b'4.566554546356201'


## Cleanup
Delete any resources you created in this notebook that you no longer wish to use.

In [21]:
client.delete_model(ModelName=model_name)
client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '1f99d6e0-d667-45f0-9483-5cc5241858aa',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1f99d6e0-d667-45f0-9483-5cc5241858aa',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Thu, 20 Mar 2025 18:22:12 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/serverless-inference|serverless-model-registry.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/serverless-inference|serverless-model-registry.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/serverless-inference|serverless-model-registry.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/serverless-inference|serverless-model-registry.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/serverless-inference|serverless-model-registry.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/serverless-inference|serverless-model-registry.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/serverless-inference|serverless-model-registry.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/serverless-inference|serverless-model-registry.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/serverless-inference|serverless-model-registry.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/serverless-inference|serverless-model-registry.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/serverless-inference|serverless-model-registry.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/serverless-inference|serverless-model-registry.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/serverless-inference|serverless-model-registry.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/serverless-inference|serverless-model-registry.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/serverless-inference|serverless-model-registry.ipynb)
